# Notebook to put all betas into individual pkl files


In [ ]:
import numpy as np
import pickle as pkl
import os
from tqdm import tqdm

## Transform NSD betas

In [ ]:
# Function that loads a pkl file with betas and stores each vector separately

def nsd_betas_to_indiv_files(betas_path, 
                         subs=None, 
                         rois=None):

    if subs is None:
        subs = sorted(os.listdir(betas_path))
    
    for sub in subs:
        # Load pickle with ids of stimuli
        with open(f'{betas_path}/{sub}/events_imgtag-73k_id.pkl', 'rb') as f:
            idxs = pkl.load(f)[0]
        print(f'Processing {sub}...')
        if rois is None:
            rois = sorted([r.split('_')[0] for r in os.listdir(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl'))])
        for roi in rois:
            roi_pkl = roi+'_betas-GLMsingle_type-typeb_z=1.pkl'
            print(f'Processing {roi_pkl}...')
            with open(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl', roi_pkl), 'rb') as f:
                data = pkl.load(f)
            d = data['data_allvoxel']
            savedir = os.path.join(betas_path, sub, 'indiv_npys', roi_pkl[:-4])
            
            # If savedir exists, delete all its contents
            if os.path.exists(savedir):
                for f in os.listdir(savedir):
                    os.remove(os.path.join(savedir, f))
            else:
                os.makedirs(savedir, exist_ok=True)

            for s, stim in tqdm(enumerate(d)):
                for r, rep in enumerate(stim):
                    npy_name = f'{idxs[s]-1:06d}_{r}.npy' #TODO: Check if indexes match correctly here
                    np.save(os.path.join(savedir, npy_name), rep)


In [ ]:
nsd_betas_to_indiv_files('../data/betas_nsd', rois=['lPPA', 'rPPA'])

## Transform BMD betas

In [ ]:


def bmd_betas_to_indiv_files(betas_path, 
                             subs=None,
                             rois=None):

    if subs is None:
        subs = sorted(os.listdir(betas_path))
    
    for sub in subs:
        print(f'Processing {sub}...')
        if rois is None:
            rois = sorted([r.split('_')[0] for r in os.listdir(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl'))])
        for roi in rois:
            roi_pkl = roi+'_betas-GLMsingle_type-typed_z=1.pkl'
            print(f'Processing {roi_pkl}...')
            with open(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl', roi_pkl), 'rb') as f:
                data = pkl.load(f)

            print(data.keys())

            # d = np.concatenate([data['train_data_allvoxel'], data['test_data_allvoxel']], axis=0)
            d_train = data['train_data_allvoxel']
            d_test = data['test_data_allvoxel']
            
            savedir = os.path.join(betas_path, sub, 'indiv_npys', roi_pkl[:-4])
            os.makedirs(savedir, exist_ok=True)
            for s, stim in enumerate(d_train):
                for r, rep in enumerate(stim):
                    npy_name = f'{s+1:04d}_{r}.npy'
                    np.save(os.path.join(savedir, npy_name), rep)
            for s, stim in enumerate(d_test):
                for r, rep in enumerate(stim):
                    npy_name = f'{s+1+len(d_train):04d}_rep{r}.npy'
                    np.save(os.path.join(savedir, npy_name), rep)


In [ ]:
bmd_betas_to_indiv_files('../data/betas_impulse', 
                     rois=['lPPA', 'rPPA'])

In [ ]:
# Transform CIFTI betas to indiv files 
# (cifti betas pickles have the same format as impulse/volumetric betas)

bmd_betas_to_indiv_files('../data/betas_cifti_bmd',
                            rois=['Group41'])
                         

In [14]:
# Rename files from 0001_0.npy to 0001_rep0.npy
# (to match the naming convention of the other datasets)

def rename_files(betas_path, 
                 subs=None,
                 rois=None):
    '''Renames individual npy files from 0001_0.npy to 0001_rep0.npy, to follow the new format.
    Only run this if your individual npy files are in the old format (e.g. 0001_0.npy for BMD)'''

    if subs is None:
        subs = sorted(os.listdir(betas_path))
    
    for sub in subs:
        if rois is None:
            rois = sorted([r.split('_')[0] for r in os.listdir(os.path.join(betas_path, sub, 'indiv_npys'))])
        for roi in rois:
            roi = roi+'_betas-GLMsingle_type-typed_z=1'
            savedir = os.path.join(betas_path, sub, 'indiv_npys', roi)
            for f in os.listdir(savedir):
                if '_rep' in f:
                    continue
                print("renaming", os.path.join(savedir, f), 'to',  os.path.join(savedir, f[:-5]+'rep'+f[-5:]))
                # os.rename(os.path.join(savedir, f),
                #         os.path.join(savedir, f[:-5]+'rep'+f[-5:]))

rename_files('../data/betas_cifti_bmd')


## Transform HAD cifti betas

In [ ]:

def had_betas_to_indiv_files(betas_path, 
                         subs=None, 
                         rois=None):
    
    if subs is None:
        subs = sorted(os.listdir(betas_path))

    for sub in subs:
        print(f'Processing {sub}...')
        if rois is None:
            rois = sorted([r.split('_')[0] for r in os.listdir(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl'))])
        for roi in rois:
            roi_pkl = roi+'_betas-GLMsingle_type-typeb_z=1.pkl'
            print(f'Processing {roi_pkl}...')
            with open(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl', roi_pkl), 'rb') as f:
                data = pkl.load(f)
            d = data['data_allvoxel']
            vid_names = data['stim_order']
            savedir = os.path.join(betas_path, sub, 'indiv_npys', roi_pkl[:-4])
            os.makedirs(savedir, exist_ok=True)
            for s, stim in enumerate(d):
                for r, rep in enumerate(stim):
                    npy_name = f'{vid_names[s]}.npy'
                    np.save(os.path.join(savedir, npy_name), rep)

In [ ]:
had_betas_to_indiv_files('../data/betas_cifti_had', 
                     rois=['Group41'])

# Transform NOD cifti Betas

In [ ]:

def nod_betas_to_indiv_files(betas_path, 
                         subs=None, 
                         rois=None):
    
    if subs is None:
        subs = sorted(os.listdir(betas_path))

    for sub in subs:
        print(f'Processing {sub}...')
        if rois is None:
            rois = sorted([r.split('_')[0] for r in os.listdir(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl'))])
        for roi in rois:
            roi_pkl = roi+'_betas_z=1.pkl'
            print(f'Processing {roi_pkl}...')
            with open(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl', roi_pkl), 'rb') as f:
                data = pkl.load(f)

            print(data['data_allvoxel'].shape) # (4000, 1, 13156)

            d = data['data_allvoxel']
            vid_names = data['stim_order']
            savedir = os.path.join(betas_path, sub, 'indiv_npys', roi_pkl[:-4])
            os.makedirs(savedir, exist_ok=True)
            for s, stim in enumerate(d):
                for r, rep in enumerate(stim):
                    npy_name = f'{vid_names[s].split("/")[-1][:-5]}.npy'
                    # print(npy_name)
                    np.save(os.path.join(savedir, npy_name), rep)


nod_betas_to_indiv_files('../data/betas_cifti_nod', 
                     rois=['Group41'],
                    #  subs=[f'sub{sub:02d}' for sub in range(1,5)],
                     )

# Transform CC2017 cifti Betas

In [8]:
def cc2017_betas_to_indiv_files(betas_path, 
                         subs=None, 
                         rois=None):
    
    # Work on all subs if subs is None
    if subs is None:
        subs = sorted(os.listdir(betas_path))

    # Loop over subjects and save npys
    for sub in subs:
        print(f'Processing {sub}...')

        # Work on all ROIs if rois is None
        if rois is None:
            rois = sorted([r.split('_')[0] for r in os.listdir(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl'))])
        
        for roi in rois:
            roi_pkl = roi+'_estimates-TSTrialEstimates_z=1.pkl'
            print(f'Processing {roi_pkl}...')
            with open(os.path.join(betas_path, sub, 'prepared_allvoxel_pkl', roi_pkl), 'rb') as f:
                data = pkl.load(f)

            savedir = os.path.join(betas_path, sub, 'indiv_npys', roi_pkl[:-4])

            # Save train npys
            rename_and_save(data['train_data_allvoxel'], data['train_stim_order'], savedir)

            # Save test npys
            rename_and_save(data['test_data_allvoxel'], data['test_stim_order'], savedir)

def rename_and_save(fmri_data, vid_names, savedir):
    os.makedirs(savedir, exist_ok=True)
            
    seen={}
    for name, stim in zip(vid_names, fmri_data):
        fixed_name = fix_seconds_format(name)
        if fixed_name in seen:
            seen[fixed_name] += 1
        else:
            seen[fixed_name] = 0
        npy_name = f'{fixed_name}_rep{seen[fixed_name]}.npy'
        # print('npy_name', npy_name)
        # print('stim.shape', stim.shape)
        # print('seen[fixed_name]', seen[fixed_name])
        np.save(os.path.join(savedir, npy_name), stim)
        

# Fix seconds format to match the .mp4 filenames, e.g. "-2" -> "-002"
import re
def fix_seconds_format(s):
    return re.sub(r'-(\d+)', lambda m: '-%03d' % int(m.group(1)), s)


cc2017_betas_to_indiv_files('../data/betas_cifti_cc2017', 
                     rois=['Group41'],
                    #  subs=[f'sub{sub:02d}' for sub in range(1,5)],
                     )

Processing sub01...
Processing Group41_estimates-TSTrialEstimates_z=1.pkl...
Processing sub02...
Processing Group41_estimates-TSTrialEstimates_z=1.pkl...
Processing sub03...
Processing Group41_estimates-TSTrialEstimates_z=1.pkl...
